<a href="https://colab.research.google.com/github/hwanys2/ai_practice/blob/main/5_3_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습ensemble learning입니다. 이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있습니다

# 1. 랜덤 포레스트
 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듭니다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듭니다.  
 예를 들어 1,000개 가방에서 100개씩 샘플을 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣습니다. 이런 식으로 계속해서 100개를 가방에서 뽑으면 중복된 샘플을 뽑을 수 있습니다. 이렇게 만들어진 샘플을 부트스트랩 샘플bootstrap sample이라고 부릅니다. 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듭니다. 1,000개 가방에서 중복하여 1,000개의 샘플을 뽑기 때문에 **부트스트랩 샘플은 훈련 세트와 크기가 같습니다**.  
 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾습니다. 분류 모델인 RandomForestClassifier는 기본적으로 **전체 특성 개수의 제곱근만큼의 특성을 선택**합니다.  
 사이킷런의 랜덤 포레스트는 기본적으로 **100개의 결정 트리를 이런 방식으로 훈련**합니다. 그다음 **분류**일 때는 **각 트리의 클래스별 확률을 평균**하여 가장 높은 확률을 가진 클래스를 예측으로 삼습니다. **회귀**일 때는 단순히 **각 트리의 예측을 평균**합니다.

In [1]:
# 데이터 분류
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
 data, target, test_size=0.2, random_state=42)

cross_validate() 함수를 사용해 교차 검증을 수행해 보겠습니다. RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어를 사용하는 것이 좋습니다. cross_validate() 함수의 n_jobs 매개변수도 -1로 지정하여 최대한 병렬로 교차 검증을 수행하겠습니다. 또 return_train_score 매개변수를 True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환합니다. 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는 데 용이합니다(return_train_score 매개변수의 기본값은 False입니다).

In [2]:
# 교차 검증을 통한 랜덤 포레스트 사용
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공합니다. criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등입니다. 또한 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산합니다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것입니다. 앞의 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 **특성 중요도**를 출력해 보겠습니다.

In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


각각 [알코올 도수, 당도, pH]였는데, 두 번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH특성의 중요도가 조금 상승했습니다. 이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문입니다. 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻습니다. 이는 **과대적합을 줄이고 일반화 성능을 높이는 데 도움**이 됩니다.

랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 
결정 트리를 훈련한다고 했습니다. 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있습니다. 이런 샘플을 OOBout of bag 샘플이라고 합니다. 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있습니다

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻었습니다. OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있습니다

#2. 엑스트라 트리
랜덤 포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점입니다. 즉 
각 결정 트리를 만들 때 전체 훈련 세트를 사용합니다. 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할합니다! 

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, 
 return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻었습니다. 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않습니다. 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 **더 많은 결정 트리를 훈련**해야 합니다. 하지만 랜덤하게 노드를 분할하기 때문에 **빠른 계산 속도**가 엑스트라 트리의 장점입니다.  
엑스트라 트리도 랜덤 포레스트와 마찬가지로 특성 중요도를 제공합니다

In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


#3. 그레이디언트 부스팅
**깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블** 하는 방법입니다. 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용합니다. 깊이가 얕은 결정 트리를 사용하기 때문에 **과대적합에 강하고 일반적으로 높은 일반화 성능을 기대**할 수 있습니다.  


In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, 
 return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


**거의 과대적합이 되지 않습니다**. 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강합니다. 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있습니다

In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, 
 return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
# 중요도
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


일반적으로 그레이디언트 부스팅이 **랜덤 포레스트보다 조금 더 높은 성능**을 얻을 수 있습니다. 하지만 순서대로 트리를 추가하기 때문에 **훈련 속도가 느립니다**. 즉 GradientBoostingClassifier에는 n_jobs 매개변수가 없습니다. 그레이디언트 부스팅의 회귀 버전은 GradientBoostingRegressor입니다. 그레이디언트 부스팅의 속도와 성능을 더욱 개선한 것이 다음에 살펴볼 히스토그램 기반 그레이디언트 부스팅입니다.

#4. 히스토그램 기반 그레이디언트 부스팅
**히스토그램 기반 그레이디언트 부스팅Histogram-based Gradient Boosting은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘입니다.**  
훈련데이터를 256개의 구간으로 나눕니다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있습니다. 히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용합니다. 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없죠!  
무슨내용인지?

In [10]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, 
 return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


0.9321723946453317 0.8801241948619236


## XGBoost와 LightGBM

In [11]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, 
 return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [13]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, 
 return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
